## Request Index page for top 500 alexa websites
- Analysis per CDN provider
    - time to first byte
    - time to DNS resolution
    - time to TCP connection
    - time to SSL negotiation
    - receive time

In [5]:
from __future__ import division
%pylab inline
import pandas as pd
import numpy as np
import subprocess
import os, sys, re
from collections import defaultdict

Populating the interactive namespace from numpy and matplotlib


In [6]:
site = 'wikipedia.org'

### CURL Command Example with Output
curl -v --trace-time -o /dev/null --connect-timeout 3.0 -w "\n\nContentType: %{content_type}\nResponseCode: %{response_code}\nURLEffective: %{url_effective}\n\nSizeDownload: %{size_download};\tSizeRquest: %{size_request}\nSizeUpload: %{size_upload};\tSizeHeader: %{size_header}\n\nTimeNameLookup: %{time_namelookup}\nTimeConnect: %{time_connect}\nTimeAppConnect: %{time_appconnect}\nTimePreTransfer: %{time_pretransfer}\nTimeRedirect: %{time_redirect}\nTimeStartTransfer: %{time_starttransfer}\nTimeTotal: %{time_total}\n" -s https://www.google.co.in


12:24:41.186600 * Rebuilt URL to: https://www.google.co.in/  
12:24:41.191017 *   Trying 172.217.167.3...  
12:24:41.191061 * TCP_NODELAY set  
12:24:41.218682 * Connected to www.google.co.in (172.217.167.3) port 443 (#0)  
12:24:41.218794 * ALPN, offering h2  
12:24:41.218825 * ALPN, offering http/1.1  
12:24:41.218890 * Cipher selection: ALL:!EXPORT:!EXPORT40:!EXPORT56:!aNULL:!LOW:!RC4:@STRENGTH  
12:24:41.224963 * successfully set certificate verify locations:  
12:24:41.224997 *   CAfile: /etc/ssl/cert.pem  
  CApath: none  
12:24:41.225187 * TLSv1.2 (OUT), TLS handshake, Client hello (1):  
12:24:41.225219 } [222 bytes data]  
12:24:41.302091 * TLSv1.2 (IN), TLS handshake, Server hello (2):  
12:24:41.302182 { [96 bytes data]  
12:24:41.302706 * TLSv1.2 (IN), TLS handshake, Certificate (11):  
12:24:41.302769 { [3128 bytes data]  
12:24:41.309234 * TLSv1.2 (IN), TLS handshake, Server key exchange (12):  
12:24:41.309275 { [116 bytes data]  
12:24:41.312864 * TLSv1.2 (IN), TLS handshake, Server finished (14):  
12:24:41.312896 { [4 bytes data]  
12:24:41.313614 * TLSv1.2 (OUT), TLS handshake, Client key exchange (16):  
12:24:41.313644 } [37 bytes data]  
12:24:41.313695 * TLSv1.2 (OUT), TLS change cipher, Client hello (1):  
12:24:41.313723 } [1 bytes data]  
12:24:41.313895 * TLSv1.2 (OUT), TLS handshake, Finished (20):  
12:24:41.313925 } [16 bytes data]  
12:24:41.318727 * TLSv1.2 (IN), TLS change cipher, Client hello (1):  
12:24:41.318799 { [1 bytes data]  
12:24:41.319048 * TLSv1.2 (IN), TLS handshake, Finished (20):  
12:24:41.319106 { [16 bytes data]  
12:24:41.319209 * SSL connection using TLSv1.2 / ECDHE-ECDSA-CHACHA20-POLY1305  
12:24:41.319265 * ALPN, server accepted to use h2  
12:24:41.319324 * Server certificate:  
12:24:41.319405 *  subject: C=US; ST=California; L=Mountain View; O=Google LLC; CN=*.google.com  
12:24:41.319470 *  start date: Nov  7 08:59:00 2018 GMT  
12:24:41.319525 *  expire date: Jan 30 08:59:00 2019 GMT  
12:24:41.319705 *  subjectAltName: host "www.google.co.in" matched cert's "*.google.co.in"  
12:24:41.319858 *  issuer: C=US; O=Google Trust Services; CN=Google Internet Authority G3  
12:24:41.319915 *  SSL certificate verify ok.  
12:24:41.319996 * Using HTTP2, server supports multi-use  
12:24:41.320050 * Connection state changed (HTTP/2 confirmed)  
12:24:41.320106 * Copying HTTP/2 data in stream buffer to connection buffer after upgrade: len=0  
12:24:41.320264 * Using Stream ID: 1 (easy handle 0x7fb146006600)  
12:24:41.320351 > GET / HTTP/2  
12:24:41.320351 > Host: www.google.co.in  
12:24:41.320351 > User-Agent: curl/7.54.0  
12:24:41.320351 > Accept: */*  
12:24:41.320351 >  
12:24:41.324419 * Connection state changed (MAX_CONCURRENT_STREAMS updated)!  
12:24:41.425512 < HTTP/2 200  
12:24:41.425586 < date: Tue, 11 Dec 2018 06:54:41 GMT  
12:24:41.425637 < expires: -1  
12:24:41.425687 < cache-control: private, max-age=0  
12:24:41.425741 < content-type: text/html; charset=ISO-8859-1  
12:24:41.425793 < p3p: CP="This is not a P3P policy! See g.co/p3phelp for more info."  
12:24:41.425845 < server: gws  
12:24:41.425897 < x-xss-protection: 1; mode=block  
12:24:41.425949 < x-frame-options: SAMEORIGIN  
12:24:41.426026 < set-cookie: 1P_JAR=2018-12-11-06; expires=Thu, 10-Jan-2019 06:54:41 GMT; path=/; domain=.google.co.in  
12:24:41.426180 < set-cookie: NID=150=pPHcNvIyWCtqTFQg8ywE9W6ZJEq7IgUcIdKsn5Od5SUcrW19sHBUblridrGt7zMn9IyRgUhJqvrJ0Hw5B9kkW51nJMsBQ9yTzwCtUYZUnGKq5E7sD5X7zNKmNXV7mmljGbb9CIqk1XigbExIVNozjA9uO_BJoPaO5gW93g_Z91I; expires=Wed, 12-Jun-2019 06:54:41 GMT; path=/; domain=.google.co.in; HttpOnly  
12:24:41.426250 < alt-svc: quic=":443"; ma=2592000; v="44,43,39,35"  
12:24:41.426303 < accept-ranges: none  
12:24:41.426377 < vary: Accept-Encoding  
12:24:41.426457 <  
12:24:41.426640 { [1388 bytes data]  
12:24:41.428591 * Connection #0 to host www.google.co.in left intact  



ContentType: text/html; charset=ISO-8859-1  
ResponseCode: 200  
URLEffective: https://www.google.co.in/  



SizeDownload: 13548;	SizeRquest: 78  
SizeUpload: 0;	SizeHeader: 759  



TimeNameLookup: 0.004405  
TimeConnect: 0.032078  
TimeAppConnect: 0.133374  
TimePreTransfer: 0.133973  
TimeRedirect: 0.000000  
TimeStartTransfer: 0.240047  
TimeTotal: 0.241995  

### Regarding curl timings from https://ec.haxx.se/usingcurl-verbose.html:  

%{time_appconnect} shows the time, in seconds, it took from the start until the SSL/SSH/etc connect/handshake to the remote host was completed.  

%{time_connect} shows the time, in seconds, it took from the start until the TCP connect to the remote host (or proxy) was completed.  

%{time_namelookup} shows the time, in seconds, it took from the start until the name resolving was completed.  

%{time_pretransfer} shows the time, in seconds, it took from the start until the file transfer was just about to begin. This includes all pre-transfer commands and negotiations that are specific to the particular protocol(s) involved.  

%{time_redirect} shows the time, in seconds, it took for all redirection steps including name lookup, connect, pre-transfer and transfer before the final transaction was started. time_redirect shows the complete execution time for multiple redirections.  

%{time_starttransfer} shows the time, in seconds, it took from the start until the first byte was just about to be transferred. This includes time_pretransfer and also the time the server needed to calculate the result.  

%{time_total} shows the total time, in seconds, that the full operation lasted. The time will be displayed with millisecond resolution.  

### Calculate Timings

T0 = timer.start()    
Tf = timer.stop()  


if not using HTTPS/SSL: time_appconnect returned is 0, so set time_appconnect = time_connect

t_dns = time for DNS resolution (no redirects) = time_namelookup - time_redirect  
t_tcp = time for TCP connection (SYN/SYNACK) = time_connect - time_namelookup  
t_ssl = time for SSL handshake (only if https) = time_appconnect - time_connect  
t_fbyte = time_starttransfer
t_wait = time between issuing GET request and first byte received = time_starttransfer - time_pretransfer  
t_rx = time to receive data from first to last byte = time_total - time_starttransfer  


t_calc = waiting time between SSL and GET request = time_pretransfer - time_appconnect  
t_start = waiting time between issuing curl command and issuing first byte of data for DNS lookup  
t_stop = waiting time between receiving last byte and ending the curl command  

In [47]:
site = 'wikipedia.org'
url = 'https://www.'+site+'/'

In [71]:
#import sys
import json
#import re

# useful when threading
#sys.stdout.write(".")
#sys.stdout.flush()

p = subprocess.Popen(['curl', '-w', '@curl_time_format.txt', '-o', '/dev/null', '-s', url], stdout=subprocess.PIPE)
out, err = p.communicate()

# convert output string to python dict using json
json.loads(out.decode('UTF-8'))

#out = re.sub(r'(\d+),(\d+)', r'\1.\2', out.decode('UTF-8'))

{'url_effective': 'https://www.wikipedia.org/',
 'response_code': 200,
 'time_namelookup': 0.004959,
 'time_connect': 0.163835,
 'time_appconnect': 0.368026,
 'time_pretransfer': 0.36818,
 'time_redirect': 0.0,
 'time_starttransfer': 0.951108,
 'time_total': 1.299881}

In [139]:
from __future__ import division
import subprocess
import json
import time
#from multiprocessing.pool import ThreadPool
#import numpy as np
#import session
import multiprocessing


list_of_websites = 'top-1m-new.csv' # location of alexa top websites as RANK,SITE\n
count = 10     # average timings over count loops of curl requests
nthreads = 2 # number of parallel threads for same url
# for each website COUNT time queries (in parallel)?
# for each result.response_code == '200' -> add to dict and find avg timings per website


def fetch_url(url):
    '''fetch url using curl
    connection timeout of 3.0 seconds
    result is [ dict {url_effective, response_code, time_namelookup, time_connect, time_appconnect,
    time_pretransfer, time_redirect, time_starttransfer, time_total}, error, exception ]
    '''
    try:
        p = subprocess.Popen(['curl', '--connect-timeout', '3.0', '-o', '/dev/null',  '-w', '@curl_time_format.txt',
                              '-s', url], stdout=subprocess.PIPE)
        out, err = p.communicate()
        result = json.loads(out.decode('UTF-8'))
        print("%r (%r) fetched with response code %r in %ss"
              % (url, result['url_effective'], result['response_code'], result['time_total']))    
    except Exception as e:
        print("Error fetching %r: Exception %s"
              % (url, e))
        result = None
    return result


def load_url_list(websites, nwebsites=500):
    urls = []
    from itertools import islice
    with open(websites) as f:
        for line in islice(f, nwebsites):
            rank, site = line.strip().split(',')
            url = 'https://www.' + site + '/'
            urls.append(url)
    return urls


In [151]:
list_of_websites = 'top-1m-new.csv'  # location of alexa top websites as RANK,SITE\n
count = 5  # average timings over count loops of curl requests
nthreads=2
data = defaultdict(list)
urls = load_url_list(list_of_websites, 5)
url_counter = 0

for url in urls:
    url_counter += 1
    # can do threading here using gevents or multiprocessing to parallelize curl requests
    urls_parallel = [url for i in range(count)]
    #print(urls_parallel)
    print("Time: %s, URL: %r" % (time.time(), url))
    
    pool = multiprocessing.Pool(processes=nthreads)
    pool_outputs = pool.map(fetch_url, urls_parallel) #pool_output is a list of results
    
    pool.close()
    pool.join()
    
    #print(pool_outputs)
    
    for res in pool_outputs:
        if res is not None:
            [data[key].append(res[key]) for key in res.keys()]
            data['rank'].append(url_counter)

Time: 1544597449.002611, URL: 'https://www.google.com/'
'https://www.google.com/' ('https://www.google.com/') fetched with response code '200' in 0.260134s
'https://www.google.com/' ('https://www.google.com/') fetched with response code '200' in 0.261776s
'https://www.google.com/' ('https://www.google.com/') fetched with response code '200' in 0.231879s
'https://www.google.com/' ('https://www.google.com/') fetched with response code '200' in 0.233389s
'https://www.google.com/' ('https://www.google.com/') fetched with response code '200' in 0.217485s
Time: 1544597449.848049, URL: 'https://www.youtube.com/'
'https://www.youtube.com/' ('https://www.youtube.com/') fetched with response code '200' in 1.850815s
'https://www.youtube.com/' ('https://www.youtube.com/') fetched with response code '200' in 2.09202s
'https://www.youtube.com/' ('https://www.youtube.com/') fetched with response code '200' in 1.096934s
'https://www.youtube.com/' ('https://www.youtube.com/') fetched with response code

In [152]:
pd.DataFrame(data)

,url_effective,response_code,time_namelookup,time_connect,time_appconnect,time_pretransfer,time_redirect,time_starttransfer,time_total,rank
0,https://www.google.com/,200,0.004907,0.013170,0.133998,0.134173,0.0,0.254098,0.260134,1
1,https://www.google.com/,200,0.004322,0.011826,0.126174,0.126342,0.0,0.259114,0.261776,1
2,https://www.google.com/,200,0.005093,0.008966,0.108502,0.108619,0.0,0.224604,0.231879,1
3,https://www.google.com/,200,0.004237,0.011423,0.108318,0.108438,0.0,0.231338,0.233389,1
4,https://www.google.com/,200,0.004396,0.014893,0.108282,0.108402,0.0,0.208904,0.217485,1
5,https://www.youtube.com/,200,0.005116,0.012317,0.123554,0.123714,0.0,0.274948,2.092020,2
6,https://www.youtube.com/,200,0.005081,0.012227,0.118582,0.118703,0.0,0.236816,1.850815,2
7,https://www.youtube.com/,200,0.004206,0.013547,0.108436,0.108612,0.0,0.215248,1.544953,2
8,https://www.youtube.com/,200,0.004324,0.008861,0.115306,0.115404,0.0,0.210809,1.096934,2
9,https://www.youtube.com/,200,0.005146,0.030249,0.137935,0.138058,0.0,0.286557,1.019667,2
